In [ ]:
using DataFrames, CSV, Statistics, Printf

# --- User input ---
file = "ud.dat"
julia_path = "./$file"
fortran_path = "/Users/jakehalpern/Github/GPEC/docs/examples/solovev_ideal_example/$file"

# --- Function to detect delimiter ---
function detect_delim(file_path)
    open(file_path) do io
        header = readline(io)
        # Count tabs vs spaces
        tab_count = count(c -> c == '\t', header)
        space_count = count(c -> c == ' ', header)
        if tab_count > space_count
            return '\t'
        else
            return ' '   # treat multiple spaces as repeated delimiter
        end
    end
end

# --- Function to read .dat file into DataFrame ---
function read_dat_auto(file_path)
    delim = detect_delim(file_path)
    df = CSV.read(file_path, DataFrame; delim=delim, ignorerepeated=true)
    return df
end

In [ ]:
julia = read_dat_auto(julia_path)
fortran = read_dat_auto(fortran_path)

# --- Check dimensions match ---
if size(julia) != size(fortran)
    error("Files have different dimensions: $(size(julia)) vs $(size(fortran))")
end

In [ ]:
function print_aligned(df_julia, df_fortran, idx)
    headers = string.(names(df_julia))
    row_j = string.(Vector(df_julia[idx, :]))
    row_f = string.(Vector(df_fortran[idx, :]))
    widths = [maximum(length.([h,vj,vf])) for (h,vj,vf) in zip(headers,row_j,row_f)]
    fmt(values,label="") = lpad(label,8) * " " *
        join([rpad(v,w) for (v,w) in zip(values,widths)], " | ")

    println(" "^8, join([rpad(h,w) for (h,w) in zip(headers,widths)], " | "))
    println(fmt(row_j,"julia:"))
    println(fmt(row_f,"fortran:"))
end

function clean_numeric!(df::DataFrame)
    for col in names(df)
        df[!, col] = [try
                          Float64(x)
                      catch
                          0.0
                      end for x in df[!, col]]
    end
    return df
end

In [ ]:
println("Column-wise statistics of Julia/Fortran differences:")
for col in names(fortran)[3:end]   # skip ipsi, jtheta
    println("=====================================")
    # Extract columns as vectors (safe for math)
    col_j = julia[!, col]
    col_f = fortran[!, col]

    # Absolute and relative differences
    col_diff = abs.(col_j .- col_f)
    rel_col_diff = abs.((col_j .- col_f) ./ col_f)
    # Locations of min and max
    max_val, max_idx = findmax(col_diff)
    rel_max_val, rel_max_idx = findmax(rel_col_diff)

    println("$col")
    println("\n  Abs Max: ", max_val, " (at row ", max_idx, ")")
    print_aligned(julia, fortran, max_idx)
    println("\n  Rel Max: ", 100*rel_max_val, "% (at row ", rel_max_idx, ")")
    print_aligned(julia, fortran, rel_max_idx)
end